In [1]:
import datetime
import numpy as np
import tensorflow as tf

2024-04-03 01:27:07.140438: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-03 01:27:07.199044: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 01:27:08.109902: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import numpy as np
# (*func7*) one_hot_encode
def one_hot_encode(tags, mapping):
    # create empty vector
    encoding = np.zeros(len(mapping), dtype='uint8')
    # mark 1 for each tag in the vector
    for tag in tags:
        encoding[mapping[tag]] = 1
    return encoding

In [3]:
# (*func6*) create_tag_mapping -> 7
# combineddata_dev = train_dev_y + test_dev_y, 'FixedByID'
def create_tag_mapping(mapping_csv, tagname):
    # tagname FixedByID
    print('tagname', tagname)
    # create a set of all known tags
    labels = set()
    IssueType_Tags = []
    for i in range(len(mapping_csv)):
        # convert spaced separated tags into an array of tags
        tags = mapping_csv[i].split('|')
        # add tags to the set of known labels
        labels.update(tags)
    # labels 存 tag(dev) 
    # convert set of labels to a list to list
    labels = list(labels)
    # order set alphabetically
    labels.sort()
    # label和数字的正反映射
    # dict that maps labels to integers, and the reverse
    labels_map = {labels[i]: i for i in range(len(labels))}
    inv_labels_map = {i: labels[i] for i in range(len(labels))}
    # mapping_csv转为onehot编码
    for i in range(len(mapping_csv)):
        # Create One Hot Encoding For Issue Type
        IssueType_Tag = one_hot_encode(mapping_csv[i].split('|'), labels_map) # (*func7*) one_hot_encode
        IssueType_Tags.append(IssueType_Tag)

    result = IssueType_Tags
    return labels_map, inv_labels_map, result

In [4]:
# (*func8*) RemoveTestRecordIfNotExistInTrainData
def RemoveTestRecordIfNotExistInTrainData(traindata, testdata):
    traingroup = traindata.groupby(["Name", "FixedByID"], as_index=True)["FixedByID"].size().reset_index(name="count")
    testgroup = testdata.groupby(["Name", "FixedByID"], as_index=True)["FixedByID"].size().reset_index(name="count")
    for ind in testgroup.index:
        try:
            record = traindata[
                traindata['FixedByID'].str.match(testgroup['FixedByID'][ind]) & traindata['Name'].str.match(
                    testgroup['Name'][ind])]
            if len(record) < 1:
                print('remove from testdata...')
                testdata = testdata.drop(testdata[
                                             testdata['FixedByID'].str.match(testgroup['FixedByID'][ind]) & testdata[
                                                 'Name'].str.match(testgroup['Name'][ind])].index)
        except:
            print("An exception occurred index :", ind)
    return testdata

In [5]:
# Setup Project Parameters
DataAugmentation, DataAugThreshold = True, 30000
DataFilePath, DataFileName, FileType = "Data/", "IssueaspnetcoreWebScrap", ".csv"
MAX_SEQUENCE_LENGTH, EMBEDDING_DIM = 300, 100
LoadDataAugFromFile = False
LearningRate = 0.001
VALIDATION_SPLIT = 0.2

In [6]:
# log file
filename = 'Multimodel' + '_' + DataFileName + '_' + "dataaug" + '_' + str(LearningRate) + '_' + str(EMBEDDING_DIM) + '_' + str(MAX_SEQUENCE_LENGTH)
filelog = open(filename + ".txt", "w")
filelog.write("StartTime:" + str(datetime.datetime.now()))
filelog.close()

In [7]:
# get traindata & testdata
import pandas as pd
traindata = pd.read_csv("./IssueaspnetcoreWebScraptrainaugdata5.csv",
                                error_bad_lines=False, index_col=False, dtype='unicode', encoding='latin-1',
                                low_memory=False).sample(frac=1)
traindata = traindata.rename(columns={'ï»¿RepoID': 'RepoID'}, inplace=False)
testdata = pd.read_csv("./IssueaspnetcoreWebScraptestdata5.csv",
                               error_bad_lines=False, index_col=False, dtype='unicode', encoding='latin-1',
                               low_memory=False).sample(frac=1)
testdata = testdata.rename(columns={'ï»¿RepoID': 'RepoID'}, inplace=False)
testdata = RemoveTestRecordIfNotExistInTrainData(traindata, testdata)

remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from testdata...
remove from test

In [8]:
# /*****************Train Data***********************/
train_dev_y = list(traindata['FixedByID'])  # Developer List
train_btype_y = list(traindata['Name'])  # Bug Type List
train_x_context = list(traindata['Title_Description'])
traindata.AST = traindata.AST.astype(str)
train_x_AST = list(traindata['AST'])

x_train_context = train_x_context
x_train_AST = train_x_AST

# /****************Test Data************************/
x_test_context = list(testdata['Title_Description'])
x_test_AST = list(testdata['Title_Description'])
test_dev_y = list(testdata['FixedByID'])  # Developer List
test_btype_y = list(testdata['Name'])  # Bug Type List

In [9]:
# /*******************Label Encoder***********************/
### Developer Encoder 
combineddata_dev = train_dev_y + test_dev_y
# (*func6*) create_tag_mapping 把train_dev_y 和 test_dev_y 由str进行onehot encoder
dev_labels_map, dev_inv_labels_map, combineddata_dev_enc = create_tag_mapping(combineddata_dev, 'FixedByID')
dev_y_train = combineddata_dev_enc[:len(train_dev_y)]
dev_y_test = combineddata_dev_enc[len(train_dev_y):]
# 31929 1430 33359
print("Developer", "Training: ", len(train_dev_y), "Testing :", len(test_dev_y), "Combined DEV + TEST",
        len(combineddata_dev_enc))

### BugType Encoder
combineddata_bugtype = train_btype_y + test_btype_y
# 同理编码bugtype
btype_labels_map, btype_inv_labels_map, combineddata_bugtype_enc = create_tag_mapping(combineddata_bugtype, 'Name')
btype_y_train = combineddata_bugtype_enc[:len(train_btype_y)]
btype_y_test = combineddata_bugtype_enc[len(train_btype_y):]
print("Bug Type", "Training: ", len(btype_y_train), "Testing :", len(btype_y_test), "Combined DEV + TEST",
        len(combineddata_bugtype_enc))

tagname FixedByID
Developer Training:  31929 Testing : 1363 Combined DEV + TEST 33292
tagname Name
Bug Type Training:  31929 Testing : 1363 Combined DEV + TEST 33292


In [10]:
pip install keras_preprocessing

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [11]:
from keras_preprocessing.text import Tokenizer
# /*******************Tokenizer****************************/

## context 小写后 转为 toeknizer 
x_train_context = [str(row).lower() for row in x_train_context]
x_test_context = [str(row).lower() for row in x_test_context]
combineddata_context = x_train_context + x_test_context
## 不限词汇表大小，以单词为单位，未知词标为Unknown，过滤掉'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\d+'
tk_context = Tokenizer(num_words=None, char_level=None, oov_token='Unknown',
                        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tk_context.fit_on_texts(combineddata_context)

## AST 小写后 转为 toeknizer 
x_train_AST = [str(row).lower() for row in x_train_AST]
x_test_AST = [str(row).lower() for row in x_test_AST]
combineddata_AST = x_train_AST + x_test_AST
tk_AST = Tokenizer(num_words=None, char_level=None, oov_token='Unknown')
tk_AST.fit_on_texts(combineddata_AST)

In [12]:
# Convert string to index 
x_train_context_sequences = tk_context.texts_to_sequences(x_train_context)
x_train_AST_sequences = tk_AST.texts_to_sequences(x_train_AST)
x_test_context_sequences = tk_context.texts_to_sequences(x_test_context)
x_test_AST_sequences = tk_AST.texts_to_sequences(x_test_AST)

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Padding
x_train_context = pad_sequences(x_train_context_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
x_train_AST = pad_sequences(x_train_AST_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
x_test_context = pad_sequences(x_test_context_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
x_test_AST = pad_sequences(x_test_AST_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# Convert to numpy array
x_train_context = np.array(x_train_context)
x_train_AST = np.array(x_train_AST)
x_test_context = np.array(x_test_context)
x_test_AST = np.array(x_test_AST)

In [14]:
noofbugtype = len(btype_labels_map)
noofdev = len(dev_labels_map)
btype_y_train = np.array(btype_y_train)
dev_y_train = np.array(dev_y_train)
btype_y_test = np.array(btype_y_test)
dev_y_test = np.array(dev_y_test)

In [15]:
# Visualize Model
logdir = "logs/"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
# A model.fit() training loop will check at end of every epoch whether the loss is no longer decreasing, considering the min_delta and patiencez
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

starttime = datetime.datetime.now()
print("Start Time =", starttime)
print('Predict Developer')

Start Time = 2024-04-03 01:27:35.638915
Predict Developer


In [16]:
from keras.layers import Input
# inputs
input_context = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.float32,
                        name="Bug_TitleandDescription")  # Bug Title and Description
input_AST = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.float32, name="Bug_CodeSnippetAST")  # Bug Code Snippet AST

In [17]:
from keras.layers import Embedding, Conv1D, GlobalMaxPool1D, Flatten
# Context Enconder
# emb_Context = Embedding(input_dim=len(tk_context.word_index) + 2, input_length=MAX_SEQUENCE_LENGTH,
emb_Context = Embedding(input_dim=len(tk_context.word_index) + 2,
                        output_dim=EMBEDDING_DIM, name="Context_Embedding")(input_context)
conv_Context = Conv1D(filters=64, kernel_size=2, padding='same', activation='relu',
                        name="Context_Convolutional_Layer")(emb_Context)
maxpool_Context = GlobalMaxPool1D(name="Context_Maxpool_Layer")(conv_Context)
flatcon = Flatten(name="Context_Flatten_Layer")(maxpool_Context)

2024-04-03 01:27:35.867005: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22290 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:d8:00.0, compute capability: 8.9


In [18]:
from keras.layers import Embedding, Bidirectional, LSTM, GlobalMaxPool1D, Flatten
# AST Enconder
# emb_AST = Embedding(input_dim=len(tk_AST.word_index) + 2, input_length=MAX_SEQUENCE_LENGTH,
emb_AST = Embedding(input_dim=len(tk_AST.word_index) + 2,
                    output_dim=EMBEDDING_DIM, name="AST_Embedding")(input_AST)
bilstm_AST = Bidirectional(LSTM(25, return_sequences=True, name="AST_LSTM_Layer"))(emb_AST)
maxpool_AST = GlobalMaxPool1D(name="AST_Maxpool_Layer")(bilstm_AST)
flatAST = Flatten(name="AST_Flatten_Layer")(maxpool_AST)

In [19]:
from keras.layers import concatenate, BatchNormalization, Dropout, Dense
from keras.models import Model
cat = concatenate([flatcon, flatAST], name="Concatenate_Flatten_Layer")

bn = BatchNormalization()(cat)
drop = Dropout(0.5)(bn)
dense = Dense(50, activation='relu')(drop)
DevOutput = Dense(noofdev, activation='sigmoid', name="Developer")(dense)
BugTypeOutput = Dense(noofbugtype, activation='sigmoid', name="Bug_Type")(dense)
Bil_LSTM_MultiTask_model = Model(inputs=[input_context, input_AST], outputs=[DevOutput, BugTypeOutput])

In [22]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import backend
def hammingloss(y_true, y_pred, threshold = 0.5, mode = 'multilabel'):
    """Computes hamming loss.
    Hamming loss is the fraction of wrong labels to the total number
    of labels.
    In multi-class classification, hamming loss is calculated as the
    hamming distance between `actual` and `predictions`.
    In multi-label classification, hamming loss penalizes only the
    individual labels.
    Args:
        y_true: actual target value
        y_pred: predicted target value
        threshold: Elements of `y_pred` greater than threshold are
            converted to be 1, and the rest 0. If threshold is
            None, the argmax is converted to 1, and the rest 0.
        mode: multi-class or multi-label
    Returns:
        hamming loss: float
    Usage:
    ```python
    # multi-class hamming loss
    hl = HammingLoss(mode='multiclass', threshold=0.6)
    actuals = tf.constant([[1, 0, 0, 0],[0, 0, 1, 0],
                       [0, 0, 0, 1],[0, 1, 0, 0]],
                      dtype=tf.float32)
    predictions = tf.constant([[0.8, 0.1, 0.1, 0],
                               [0.2, 0, 0.8, 0],
                               [0.05, 0.05, 0.1, 0.8],
                               [1, 0, 0, 0]],
                          dtype=tf.float32)
    hl.update_state(actuals, predictions)
    print('Hamming loss: ', hl.result().numpy()) # 0.25
    # multi-label hamming loss
    hl = HammingLoss(mode='multilabel', threshold=0.8)
    actuals = tf.constant([[1, 0, 1, 0],[0, 1, 0, 1],
                       [0, 0, 0,1]], dtype=tf.int32)
    predictions = tf.constant([[0.82, 0.5, 0.90, 0],
                               [0, 1, 0.4, 0.98],
                               [0.89, 0.79, 0, 0.3]],
                               dtype=tf.float32)
    hl.update_state(actuals, predictions)
    print('Hamming loss: ', hl.result().numpy()) # 0.16666667
    ```
    """
    if mode not in ["multiclass", "multilabel"]:
        raise TypeError("mode must be either multiclass or multilabel]")

    if threshold is None:
        threshold = tf.reduce_max(y_pred, axis=-1, keepdims=True)
        # make sure [0, 0, 0] doesn't become [1, 1, 1]
        # Use abs(x) > eps, instead of x != 0 to check for zero
        y_pred = tf.logical_and(y_pred >= threshold, tf.abs(y_pred) > 1e-12)
    else:
        y_pred = y_pred > threshold

    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.int32)

    if mode == "multiclass":
        nonzero = tf.cast(tf.math.count_nonzero(y_true * y_pred, axis=-1), tf.float32)
        return 1.0 - nonzero

    else:
        nonzero = tf.cast(tf.math.count_nonzero(y_true - y_pred, axis=-1), tf.float32)
        return nonzero / y_true.get_shape()[-1]

def c_precision(y_true, y_pred):
    '''Calculates the precision, a metric for multi-label classification of
    how many selected items are relevant.
    '''
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    true_positives = tf.cast(K.sum(K.round(K.clip(y_true * y_pred, 0, 1))), tf.float32)
    predicted_positives = tf.cast(K.sum(K.round(K.clip(y_pred, 0, 1))), tf.float32)
    precision = tf.cast(true_positives / (predicted_positives + K.epsilon()), tf.float32)
    return precision

def c_recall(y_true, y_pred):
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    '''Calculates the recall, a metric for multi-label classification of
    how many relevant items are selected.
    '''
    true_positives = tf.cast(K.sum(K.round(K.clip(y_true * y_pred, 0, 1))), tf.float32)
    possible_positives = tf.cast(K.sum(K.round(K.clip(y_true, 0, 1))), tf.float32)
    recall = tf.cast(true_positives / (possible_positives + K.epsilon()), tf.float32)
    return recall
    
def c_fbeta(y_true, y_pred, beta=1):
    y_pred = backend.clip(y_pred, 0, 1)
    # calculate elements
    tp = backend.sum(backend.round(y_pred) * y_true)
    fp = backend.sum(backend.round(y_pred) * (1 - y_true))
    fn = backend.sum((1 - backend.round(y_pred)) * y_true)
    # tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
    # fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
    # fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
    # calculate precision
    p = tp / (tp + fp + backend.epsilon())
    # calculate recall
    r = tp / (tp + fn + backend.epsilon())
    # calculate fbeta, averaged across each class
    bb = beta ** 2
    fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
    return fbeta_score

def c_auroc(y_true, y_pred):
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    auc = tf.cast(tf.metrics.auc(y_true, y_pred)[1], tf.float32)
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [23]:
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
Bil_LSTM_MultiTask_model.compile(loss='binary_crossentropy', optimizer=Adam(LearningRate),
                                metrics=[
                                    ['accuracy', c_fbeta], 
                                    ['accuracy', c_fbeta]
                                ])

history = Bil_LSTM_MultiTask_model.fit([x_train_context, x_train_AST],
                                        [backend.cast(dev_y_train, 'float32'), backend.cast(btype_y_train, 'float32')],
                                        callbacks=[tensorboard_callback, earlystop],
                                        epochs=50, verbose=2, validation_split=0.1)

Epoch 1/50


2024-04-03 01:29:03.512741: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


898/898 - 24s - 27ms/step - Bug_Type_accuracy: 0.0697 - Bug_Type_c_fbeta: 0.0085 - Developer_accuracy: 0.4128 - Developer_c_fbeta: 0.2722 - loss: 0.1595 - val_Bug_Type_accuracy: 0.1084 - val_Bug_Type_c_fbeta: 0.0304 - val_Developer_accuracy: 0.5017 - val_Developer_c_fbeta: 0.4528 - val_loss: 0.0901
Epoch 2/50
898/898 - 18s - 20ms/step - Bug_Type_accuracy: 0.1599 - Bug_Type_c_fbeta: 0.1014 - Developer_accuracy: 0.5536 - Developer_c_fbeta: 0.4913 - loss: 0.0879 - val_Bug_Type_accuracy: 0.2164 - val_Bug_Type_c_fbeta: 0.1935 - val_Developer_accuracy: 0.7410 - val_Developer_c_fbeta: 0.6917 - val_loss: 0.0699
Epoch 3/50
898/898 - 17s - 19ms/step - Bug_Type_accuracy: 0.2078 - Bug_Type_c_fbeta: 0.1924 - Developer_accuracy: 0.6678 - Developer_c_fbeta: 0.6309 - loss: 0.0727 - val_Bug_Type_accuracy: 0.2678 - val_Bug_Type_c_fbeta: 0.3027 - val_Developer_accuracy: 0.8193 - val_Developer_c_fbeta: 0.8143 - val_loss: 0.0622
Epoch 4/50
898/898 - 17s - 19ms/step - Bug_Type_accuracy: 0.2337 - Bug_Type_c_

In [27]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger(filename + '.csv', append=True, separator=';')
Bil_LSTM_MultiTask_model.evaluate([x_test_context, x_test_AST], 
                                  [backend.cast(dev_y_test, 'float32'), backend.cast(btype_y_test, 'float32')], callbacks=[csv_logger])


43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - Bug_Type_accuracy: 0.1776 - Bug_Type_c_fbeta: 0.2048 - Developer_accuracy: 0.6799 - Developer_c_fbeta: 0.6627 - loss: 0.1423


[0.14306551218032837,
 0.15920762717723846,
 0.1890958696603775,
 0.6735143065452576,
 0.6577615141868591]

In [28]:
y_pred_list = Bil_LSTM_MultiTask_model.predict([x_test_context, x_test_AST])

43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [36]:
np.round(y_pred_list[0][0]), np.round(y_pred_list[1][0])

(array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0.,

In [37]:
(dev_y_test[0]), (btype_y_test[0])

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=uint8),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8))

In [40]:
c_fbeta(y_pred_list[0][0], tf.cast(dev_y_test[0], float))

<tf.Tensor: shape=(), dtype=float32, numpy=0.05310153>

In [41]:
c_fbeta(y_pred_list[1][0], tf.cast(btype_y_test[0], float))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0004939954>